In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'vegetable/train'
test_dir = 'vegetable/test'
validation_dir = 'vegetable/validation'

#---Nuotrauku paruosimas
train_datagen = ImageDataGenerator(
    rescale = 1./255,      # 255 maximali reiksme RGB spalvu skaleja
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range =0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical'
)

test_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical'
)

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [16]:
from tensorflow.keras import datasets, layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (150,150,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation = 'relu' ),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation = 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(15, activation='softmax')
])

c:\Users\sarun\OneDrive\Documents\Code\Testinei\mokymai\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer = Adam(learning_rate=0.0001), # mokimosi greitis
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [18]:
history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

model.save('vegetables.h5')

Epoch 1/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 89s 188ms/step - accuracy: 0.2806 - loss: 2.1702 - val_accuracy: 0.6079 - val_loss: 1.2200
Epoch 2/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 0s 85us/step - accuracy: 0.5312 - loss: 1.3112 - val_accuracy: 0.7500 - val_loss: 0.8587
Epoch 3/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 134s 284ms/step - accuracy: 0.5897 - loss: 1.2480 - val_accuracy: 0.7238 - val_loss: 0.8566
Epoch 4/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 0s 62us/step - accuracy: 0.5938 - loss: 1.2668 - val_accuracy: 0.7500 - val_loss: 0.7182
Epoch 5/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 211s 450ms/step - accuracy: 0.6854 - loss: 0.9700 - val_accuracy: 0.7789 - val_loss: 0.7004
Epoch 6/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 0s 70us/step - accuracy: 0.8125 - loss: 0.5578 - val_accuracy: 0.6667 - val_loss: 0.7488
Epoch 7/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 171s 364ms/step - accuracy: 0.7321 - loss: 0.8292 - val_accuracy: 0.8175 - val_loss: 0.5799
Epoch 8/20
468/468 ━━━━━━━━━━━━━━━━━━━━ 1s 442us/step - accuracy: 0.8438 - loss: 0.475

In [40]:
import numpy as np
from PIL import Image

def preprocess_image(image_path):
    img = Image.open(image_path)
    if image_path.endswith(".png"):
        im = Image.open(image_path)
        img = im.convert('RGB')
    img = img.resize((150,150))
    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

class_indices = train_generator.class_indices
class_labels = {value: key for key, value in class_indices.items()}


def predict_vegetable(path_to_img):
    img_array = preprocess_image(path_to_img)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    return class_labels[predicted_class[0]]

print(predict_vegetable('morka.jpg'))
print(predict_vegetable('morka2.png'))
print(predict_vegetable('paprika.jpg'))
print(predict_vegetable('paprika2.jpg'))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Tomato
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Potato
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Capsicum
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Capsicum


In [42]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(test_loss)
print(test_accuracy)

93/93 ━━━━━━━━━━━━━━━━━━━━ 9s 91ms/step - accuracy: 0.8942 - loss: 0.3248
0.3310847878456116
0.8914650678634644
